In [2]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy import stats
import scipy
import csv
import pandas as pd
import math
import statsmodels
import networkx as nx
import itertools
import seaborn as sns
import random
import os
from textwrap import wrap
import pickle
from sklearn.model_selection import train_test_split, cross_validate
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [34]:
df = pd.read_csv('data/gwas/gwas_catalog_v1.0.2-associations_e96_r2019-11-21.tsv', sep='\t')

In [35]:
gwas11 = df[df['CHR_ID']=='11']

In [37]:
gwas11.head(5)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
30,2018-10-10,30054594,MacGregor S,2018-07-27,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/30054594,Genome-wide association study of intraocular p...,Intraocular pressure,"133,492 European ancestry individuals",NaN,...,7.698970,NaN,0.0735,[0.048-0.099] unit increase,"Affymetrix, Illumina [up to 40000000] (imputed)",N,intraocular pressure measurement,http://www.ebi.ac.uk/efo/EFO_0004695,GCST006394,Genome-wide genotyping array
66,2018-10-10,30054594,MacGregor S,2018-07-27,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/30054594,Genome-wide association study of intraocular p...,Glaucoma,"11,018 European ancestry cases, 126,068 Europe...",NaN,...,5.397940,NaN,1.08992,[1.05069095486461-1.13060399638518],Affymetrix [up to 40000000] (imputed),N,glaucoma,http://www.ebi.ac.uk/efo/EFO_0000516,GCST006395,Genome-wide genotyping array
78,2018-10-10,30054594,MacGregor S,2018-07-27,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/30054594,Genome-wide association study of intraocular p...,Glaucoma,"11,018 European ancestry cases, 126,068 Europe...",NaN,...,11.698970,NaN,1.17647,[1.12-1.22],Affymetrix [up to 40000000] (imputed),N,glaucoma,http://www.ebi.ac.uk/efo/EFO_0000516,GCST006395,Genome-wide genotyping array
92,2018-10-10,30054594,MacGregor S,2018-07-27,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/30054594,Genome-wide association study of intraocular p...,Glaucoma,"11,018 European ancestry cases, 126,068 Europe...",NaN,...,10.698970,NaN,1.14431,[1.10010320705662-1.19028883957134],Affymetrix [up to 40000000] (imputed),N,glaucoma,http://www.ebi.ac.uk/efo/EFO_0000516,GCST006395,Genome-wide genotyping array
96,2018-10-10,30054594,MacGregor S,2018-07-27,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/30054594,Genome-wide association study of intraocular p...,Glaucoma,"11,018 European ancestry cases, 126,068 Europe...",NaN,...,7.522879,NaN,1.09494,[1.06030436107629-1.13070801673771],Affymetrix [up to 40000000] (imputed),N,glaucoma,http://www.ebi.ac.uk/efo/EFO_0000516,GCST006395,Genome-wide genotyping array


In [42]:
snps = []
for snp in gwas11['CHR_POS'].values:
    snp_id = int(snp)
    snps.append('chr11:{}-{}'.format(snp_id, snp_id+1))
    
f= open("snps38.txt","w+")
for i in snps:
     f.write("{}\n".format(i))

In [43]:
gwas11['CHR_POS'].values

array(['119346601', '86695117', '120377784', ..., '65564802', '99282070',
       '114140943'], dtype=object)

In [28]:
snps

['chr1:883245-883246',
 'chr1:746491-746492',
 'chr1:58073046-58073047',
 'chr1:11217878-11217879',
 'chr1:7924522-7924523',
 'chr1:683686-683687',
 'chr1:10897164-10897165',
 'chr1:3781962-3781963',
 'chr1:4141194-4141195',
 'chr1:10767734-10767735',
 'chr1:2697920-2697921',
 'chr1:7123436-7123437',
 'chr1:746491-746492',
 'chr1:11217878-11217879',
 'chr1:7924522-7924523',
 'chr1:11040813-11040814',
 'chr1:729210-729211',
 'chr1:73034263-73034264',
 'chr1:7126821-7126822',
 'chr1:1868912-1868913',
 'chr1:7928073-7928074',
 'chr1:483450-483451',
 'chr1:12807250-12807251',
 'chr1:113003693-113003694',
 'chr1:4939208-4939209',
 'chr1:58969255-58969256',
 'chr1:801742-801743',
 'chr1:2291467-2291468',
 'chr1:59311588-59311589',
 'chr1:10792472-10792473',
 'chr1:74443493-74443494',
 'chr1:34226552-34226553',
 'chr1:7934386-7934387',
 'chr1:2155243-2155244',
 'chr1:11215126-11215127',
 'chr1:10893002-10893003',
 'chr1:1939800-1939801',
 'chr1:2716036-2716037',
 'chr1:7111153-7111154',
 'chr

In [44]:
hg19_snps = []
with open("hglft_genome_a57b_c5fc10.bed") as f: 
    for line in f:
        chrom, start, end = line.strip().replace('-', ':').split(":")
        hg19_snps.append(start)

In [45]:
hg19_snps

['119217311',
 '86406159',
 '120248493',
 '120340383',
 '128380742',
 '63892301',
 '61277775',
 '1907609',
 '17011176',
 '28642381',
 '47370607',
 '48013484',
 '86406159',
 '120340383',
 '128380742',
 '6272677',
 '41259180',
 '133808038',
 '43046643',
 '45453212',
 '47361857',
 '56908956',
 '57600285',
 '57896733',
 '58126305',
 '58822334',
 '65914766',
 '68216756',
 '72435996',
 '79905972',
 '84897574',
 '90409355',
 '91509923',
 '93308188',
 '96636768',
 '99127252',
 '105814770',
 '109636727',
 '112834153',
 '113299910',
 '119440066',
 '119640427',
 '120423601',
 '120524884',
 '121811430',
 '121983370',
 '126978142',
 '132701183',
 '72462634',
 '76477244',
 '79129615',
 '79905972',
 '84841573',
 '85265779',
 '85869737',
 '89968246',
 '90393587',
 '91509923',
 '133548873',
 '133808038',
 '112910783',
 '113299829',
 '113724640',
 '119440066',
 '120441647',
 '121784614',
 '121983370',
 '126965995',
 '131289820',
 '132702488',
 '91903709',
 '93308188',
 '95544391',
 '95834689',
 '9914081